<a href="https://colab.research.google.com/github/1kaiser/test2023/blob/main/NDSIcalculationGDAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**file inputs downloading**

In [ ]:
import tensorflow as tf
import pathlib
dir='/content/files'
urls = 'https://github.com/1kaiser/Snow-cover-area-estimation/releases/download/v1/imagesfolder.zip'
data_dir = tf.keras.utils.get_file(origin=urls,
                                   fname='s',
                                   cache_subdir= dir,
                                   archive_format='auto',
                                   untar=False,
                                   extract=True)
!rm -r {dir}/s
data_dir = pathlib.Path(data_dir)

**file listing from folder**

In [ ]:
image_dir = r'/content/files/'

#############################################################################
prefix = "sur_refl_"
end = ["b01", "b02", "b03", "b04", "b05", "b06", "b07", "day_of_year", "qc_500m", "raz", "state_500m", "szen", "vzen"]
DayOY = "_doy\[0-9]+_aid0001"
fileExt = r'.tif'
expression_b1 = prefix+end[0]
expression_b2 = prefix+end[1]
expression_b3 = prefix+end[2]
expression_b4 = prefix+end[3]
expression_b5 = prefix+end[4]
expression_b6 = prefix+end[5]
expression_b7 = prefix+end[6]

import os
imgs_list_b1 = [f for f in os.listdir(image_dir) if f.__contains__(expression_b1)]

imgs_list_b1.sort(reverse=True)       

**gdal NDSI generation**

In [ ]:
from google.colab import output
from osgeo import gdal

temp_dir = r'/content/out/'
!mkdir -p {temp_dir}
def ybatchedimages(images_path, image_list, batch_idx):
  images = []
  path = os.path.join(images_path, image_list[batch_idx])
  pathb2 = path.replace(expression_b1, expression_b2)
  pathb4 = path.replace(expression_b1, expression_b4)
  pathb6 = path.replace(expression_b1, expression_b6)
  outpath = image_list[batch_idx].replace(expression_b1, "bxxx")

  #creating file NDSI
  !gdal_calc.py \
    --overwrite \
    --type=Float32 \
    --NoDataValue=0 \
    -A {pathb4} \
    --A_band 1 \
    -B {pathb6} \
    --B_band 1 \
    -C {pathb2} \
    --C_band 1 \
    --outfile={temp_dir}{outpath} \
    --calc="(((A.astype(float) - B)/(A.astype(float) + B))>=0.4)*(C.astype(float)/10000>0.11)"
  print("<<calculation complete ✅🔻>>")
  # pathout = temp_dir+str('BothCheck_result_final.tif')
  # images.append(normalize(tif2array(pathout, 0)[0]))
  
  # !rm -r {temp_dir}"BothCheck_result_final.tif"
  output.clear()
  # return images

**run>>>**

In [ ]:
batch_size = 1
no_of_batches = int(len(imgs_list_b1)/batch_size)
for i in range(no_of_batches):
  ybatchedimages(image_dir, imgs_list_b1, i)

**zipping**

In [ ]:
%cd {temp_dir}
%cd ..
!zip -r folder.zip {temp_dir}